In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from IPython import display
from time import perf_counter
from tqdm.notebook import tqdm
import os

os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private'

gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    
keras.mixed_precision.set_global_policy('mixed_float16')

In [ ]:
def test_model(model, length, mode='random', index=4):
    if mode == 'random':
        predictions = model.random_generate(length**2)
        plt.figure(figsize=(length, length))
        for i in range(length**2):
            plt.subplot(length, length, i+1)
            plt.imshow(predictions[i], cmap='gray')
            plt.axis('off')
    if mode == 'categorical':
        predictions = model.categorical_generate(length)
        plt.figure(figsize=(10, length))
        for i in range(10*length):
            plt.subplot(length, 10, i+1)
            plt.imshow(predictions[i], cmap='gray')
            plt.axis('off')
    if mode == 'interpolated':
        predictions = model.interpolated_generate(length**2, digit=index)
        plt.figure(figsize=(length, length))
        for i in range(length**2):
            plt.subplot(length, length, i+1)
            plt.imshow(predictions[i], cmap='gray')
            plt.axis('off')
    plt.tight_layout()
    plt.show()

def show_history(history):
    plt.plot(history['disc_loss'], label='Discriminator Loss')
    plt.plot(history['gen_loss'], label='Generator Loss')
    plt.title('Learning Curve')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()

In [ ]:
class InfoGAN:
    def __init__(self, disc, gen, disc_opt, gen_opt, z_dim=64):
        self.disc = disc
        self.gen = gen
        self.disc_opt = disc_opt
        self.gen_opt = gen_opt
        self.z_dim = z_dim
        self.BCE_loss = keras.losses.BinaryCrossentropy(from_logits=True)
        self.CCE_loss = keras.losses.CategoricalCrossentropy(from_logits=True)
        self.MSE_loss = keras.losses.MeanSquaredError()
        self.history = {'disc_loss':[], 'gen_loss':[]}
        self.disc_mean_loss = keras.metrics.Mean(name='disc_loss')
        self.gen_mean_loss = keras.metrics.Mean(name='gen_loss')
    
    def random_generate(self, length):
        noise = tf.random.normal([length**2, self.z_dim])
        c_labels = tf.random.uniform([length**2], minval=0, maxval=10, dtype=tf.int32)
        c_one_hot_labels = tf.one_hot(c_labels, 10)
        c_features = tf.random.normal([batch_size, 2])
        noise_and_labels = tf.concat((noise, c_one_hot_labels, c_features), axis=-1)
        return (self.gen(noise_and_labels, training=False).numpy().reshape(-1, 28, 28) + 1.) / 2
    
    def categorical_generate(self, n_each):
        noise = tf.random.normal([n_each*10, self.z_dim])
        c_labels = tf.tile(tf.range(10), [n_each])
        c_one_hot_labels = tf.one_hot(c_labels, 10)
        c_features = tf.random.normal([n_each*10, 2])
        noise_and_labels = tf.concat((noise, c_one_hot_labels, c_features), axis=-1)
        return (self.gen(noise_and_labels, training=False).numpy().reshape(-1, 28, 28) + 1.) / 2
    
    def interpolated_generate(self, length, digit=4):
        noise = tf.tile(tf.random.normal([1, self.z_dim]), [length**2, 1])
        c_one_hot_labels = tf.tile(tf.one_hot([digit], 10), [length**2, 1])
        inter = tf.cast(tf.expand_dims(tf.linspace(-3, 3, length), axis=-1), tf.float32)
        noise_and_labels = tf.concat((noise, c_one_hot_labels, tf.tile(inter, [length, 1]), tf.repeat(inter, length, axis=0)), axis=-1)
        return (self.gen(noise_and_labels, training=False).numpy().reshape(-1, 28, 28) + 1.) / 2
    
    @tf.function(jit_compile=True)
    def train_step(self, real):
        batch_size = real.shape[0]
        noise = tf.random.normal([batch_size, self.z_dim])
        c_labels = tf.random.uniform([batch_size], minval=0, maxval=10, dtype=tf.int32)
        c_one_hot_labels = tf.one_hot(c_labels, 10)
        c_features = tf.random.normal([batch_size, 2])
        noise_and_labels = tf.concat((noise, c_one_hot_labels, c_features), axis=-1)
        fake = self.gen(noise_and_labels, training=False)
        concatenated = tf.concat((fake, real), axis=0)
        with tf.GradientTape() as disc_tape:
            disc_concat_pred, disc_concat_approx = self.disc(concatenated)
            concatenated_labels = tf.concat((tf.zeros([batch_size, 1]), tf.ones([batch_size, 1])), axis=0)
            disc_loss = self.BCE_loss(concatenated_labels, disc_concat_pred)
            category_loss = self.CCE_loss(c_one_hot_labels, disc_concat_approx[:batch_size, :10])
            feature_loss = self.MSE_loss(c_features, disc_concat_approx[:batch_size, 10:])
            total_disc_loss = disc_loss + category_loss + feature_loss
        grad_of_disc = disc_tape.gradient(total_disc_loss, self.disc.trainable_variables)
        self.disc_opt.apply_gradients(zip(grad_of_disc, self.disc.trainable_variables))
        self.disc_mean_loss.update_state(total_disc_loss)
        
        noise = tf.random.normal([batch_size, self.z_dim])
        c_labels = tf.random.uniform([batch_size], minval=0, maxval=10, dtype=tf.int32)
        c_one_hot_labels = tf.one_hot(c_labels, 10)
        c_features = tf.random.normal([batch_size, 2])
        noise_and_labels = tf.concat((noise, c_one_hot_labels, c_features), axis=-1)
        with tf.GradientTape() as gen_tape:
            fake = self.gen(noise_and_labels)
            disc_fake_pred, disc_approx = self.disc(fake, training=False)
            gen_loss = self.BCE_loss(tf.ones_like(disc_fake_pred), disc_fake_pred)
            category_loss = self.CCE_loss(c_one_hot_labels, disc_approx[:, :10])
            feature_loss = self.MSE_loss(c_features, disc_approx[:, 10:])
            total_gen_loss = gen_loss + category_loss + feature_loss
        grad_of_gen = gen_tape.gradient(total_gen_loss, self.gen.trainable_variables)
        self.gen_opt.apply_gradients(zip(grad_of_gen, self.gen.trainable_variables))
        self.gen_mean_loss.update_state(total_gen_loss)
    
    def fit(self, data, epochs=1):
        for epoch in range(epochs):
            tic = perf_counter()
            self.disc_mean_loss.reset_states()
            self.gen_mean_loss.reset_states()
            for real in tqdm(data):
                model.train_step(real)
            self.history['disc_loss'].append(self.disc_mean_loss.result().numpy())
            self.history['gen_loss'].append(self.gen_mean_loss.result().numpy())
            display.clear_output(wait=True)
            print("Epoch %d/%d - %.1fs | disc_loss: %.5f - gen_loss: %.5f"%(
                epoch+1, EPOCHS, perf_counter() - tic, self.history['disc_loss'][-1], self.history['gen_loss'][-1]))
            test_model(self, 4, mode='categorical')
            show_history(self.history)
        return self.history

In [ ]:
TRAIN_BUFFER=70000
BATCH_SIZE=64
EPOCHS = 100
DIMS = (28, 28, 1)
AUTOTUNE = tf.data.AUTOTUNE

z_dim = 64
c_dim = 12
lr = 2e-4
beta_1 = 0.5
beta_2 = 0.999

In [ ]:
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
X = np.concatenate((x_train, x_test)).astype(np.float32).reshape(-1, 28, 28, 1) / 127.5 - 1.
dataloader = tf.data.Dataset.from_tensor_slices(X).shuffle(TRAIN_BUFFER).batch(
    BATCH_SIZE, num_parallel_calls=AUTOTUNE, deterministic=False, drop_remainder=True).prefetch(AUTOTUNE)

inputs = keras.Input(DIMS)
t = layers.Conv2D(32, 4, strides=2, padding='same', use_bias=False)(inputs)
t = layers.BatchNormalization()(t)
t = layers.LeakyReLU(alpha=0.2)(t)

t = layers.Conv2D(64, 4, strides=2, padding='same', use_bias=False)(t)
t = layers.BatchNormalization()(t)
t = layers.LeakyReLU(alpha=0.2)(t)

t = layers.Conv2D(128, 4, strides=2, padding='same', use_bias=False)(t)
t = layers.BatchNormalization()(t)
t = layers.LeakyReLU(alpha=0.2)(t)
t = layers.Flatten()(t)

classification = layers.Dense(1, dtype=tf.float32)(t)
approximation = layers.Dense(c_dim, dtype=tf.float32)(t)
discriminator = keras.Model(inputs, [classification, approximation])

generator = keras.Sequential([
    layers.InputLayer([z_dim+c_dim]),
    layers.Dense(7*7*128, use_bias=False),
    layers.Reshape([7, 7, 128]),
    layers.BatchNormalization(),
    layers.ReLU(),

    layers.Conv2DTranspose(64, 4, strides=2, padding='same', use_bias=False),
    layers.BatchNormalization(),
    layers.ReLU(),

    layers.Conv2DTranspose(32, 4, strides=2, padding='same', use_bias=False),
    layers.BatchNormalization(),
    layers.ReLU(),

    layers.Conv2D(1, 3, strides=1, padding='same', activation='tanh', dtype=tf.float32)
])

model = InfoGAN(
    disc=discriminator,
    gen=generator,
    disc_opt=keras.optimizers.Adam(learning_rate=lr, beta_1=beta_1, beta_2=beta_2),
    gen_opt=keras.optimizers.Adam(learning_rate=lr, beta_1=beta_1, beta_2=beta_2),
    z_dim=z_dim
)

In [ ]:
history = model.fit(dataloader, epochs=EPOCHS)

In [ ]:
test_model(model, 8, mode='interpolated', index=3)

In [ ]:
#model_path = "./InfoGAN"
#model.gen.save(model_path, include_optimizer=False)

In [ ]:
"""model_path = "./InfoGAN"
loaded_model = InfoGAN(
    gen=keras.models.load_model(model_path, compile=False),
    disc=None,
    gen_opt=None,
    disc_opt=None,
    z_dim=64
)"""

In [ ]:
#test_model(loaded_model, 8)